In [6]:
from config import parameters
import pickle, csv, glob, os
import numpy as np

def load_pkl(filepath):
    with open(filepath, 'rb') as f:
        loaded_item = pickle.load(f)
    print('Loaded %s' % filepath)
    return loaded_item

def start_csv(filepath):
    f = open(filepath, 'w', encoding='utf-8-sig')
    wr = csv.writer(f)
    return f, wr

def end_csv(f, filepath):
    f.close()
    print('Created %s' % filepath)

In [2]:
output_final_report_csv_filepath = parameters.output_final_report_csv_filepath
output_raw_df_pkl_filepath = parameters.output_raw_df_pkl_filepath
output_test_dir = parameters.output_test_dir

In [ ]:
def fill_kfold_results(kfold_results, domains):
    for domain in domains:
        filepaths = glob.glob(os.path.join(output_test_dir, '[%s*.csv'%domain))
        for filepath in filepaths:
            df = pd.read_csv(filepath)
            for measure in ['F1 score (multiple)', 'F1 score (distinct)']:
                for all_or_subset in ['All', 'Best subset']:
                    kfold_results['_'.join([domain, measure, all_or_subset])].append(df[df['Measure']==measure][all_or_subset].values[0])

raw_df = load_pkl(output_raw_df_pkl_filepath)
domains = raw_df['domain'].unique()

kfold_results = defaultdict(lambda: [])

fill_kfold_results(kfold_results, domains)

f, wr = start_csv(output_final_report_csv_filepath)
wr.writerow(['Domain', 'Measure', 'All', 'Best subset'])
for domain in domains:
    for measure in ['F1 score (multiple)', 'F1 score (distinct)']:
        wr.writerow([domain, measure, '%.4f'%np.mean(kfold_results['_'.join([domain, measure, 'All'])]), '%.4f'%np.mean(kfold_results['_'.join([domain, measure, 'Best subset'])])])
end_csv(f, output_final_report_csv_filepath)